## Main

- Imagine you have 111000

- The number of type 2 operations to get alternating string is the number of positions that don't match when comparing `s` to both '101010' and '010101'
    - `111000` vs `101010`
        - 1 and 1: match
        - 1 and 0: no match
        - 1 and 0: match
        - 0 and 0: match
        - 0 and 1: no match
        - 0 and 0: match
        - represent this as '010010', 2 flips needed
    - `111000` vs `010101`
        - 1 and 0: no match
        - 1 and 1: match
        - 1 and 0: no match
        - 0 and 1: no match
        - 0 and 0: match
        - 0 and 1: no match
        - represent this as '101101', 4 flips needed
    
    - Set minops to 2 (i.e. number of flips needed to go from 111000 to 101010)

- Then, imagine if we perform one type 1 operation on the first character, so `s` becomes `110001`
    - `110001` vs `101010`
        - 1 and 1: match
        - 1 and 0: no match
        - 0 and 1: no match
        - 0 and 0: match
        - 0 and 1: no match
        - 1 and 0: no match
        - 4 flips needed
    - `110001` vs `010101`
        - 1 and 0: no match
        - 1 and 1: match
        - 0 and 0: match
        - 0 and 1: no match
        - 0 and 0: match
        - 1 and 1: match
        - 2 flips needed

- If we keep this up, we can do `n-1` type-1 operations before we get back the original string, and for each string, the comparison takes $O(N)$ time
    - This gives us time complexity of $O(N^2)$

- But is it really $O(N^2)$?
    - Consider `111000` 
    - Let's ignore the first character (which we are popping to the back)
    - For the remaining 5 characters `11000`, we are comparing it with `10101` and `01010`
    - AFTER popping the first character to the end, the `11000` does not change! So the resulting count of mismatches should also not change
    - In fact, after we apply type 1 operation, all we really need to do is to add 1 to the count of bit flips if the last character does not match the target

- Concretely
    - `111000` vs `101010`: 2 flips, 4 match
        - Pre-swap, left-most character match. 
            - Removing leftmost character leads to comparing `11000` with `01010`. We don't have to iterate again; since the left-most character matches pre-flip, removing it must lead to 2 flips 3 match for the substring.
        - Post-swap, right-most character becomes `1`, so `s = 110001`
            - Since our intermediate comparison is against `01010`, we compare the last character against `010101` in the end-state
            - The last character matches, so 2 flips 3 match becomes 2 flips 4 match
            - Checking against our manual loop, this gives us the same result!
    - `111000` vs `010101`: 4 flips, 2 match
        - Pre-swap, left-most character don't match. 
            - Removing leftmost character leads to comparing `11000` with `10101`. We don't have to iterate again; since the left-most character does not match pre-swap, removing it must lead to 3 flips 2 match for the substring.
        - Post-swap, right-most character becomes `1`, so `s = 110001`
            - Since our intermediate comparison is against `10101`, we compare the last character against `101010` in the end-state
            - The last character does not match, so 3 flips 2 match becomes 4 flips 2 match
            - Checking against our manual loop, this gives us the same result!

- We still need to consider $N$ possible strings, but each string only needs $O(1)$ operations since we don't actually need to loop over the entire string

- Therefore, solution is $O(N)$!

### Your better implementation

In [92]:
class Solution:
    def minFlips(self, s: str) -> int:
        ## Concat s with itself, to avoid messy string manipulations later
        strlen = len(s)
        s = s + s

        target1 = '01' * (len(s)//2)
        target2 = '10' * (len(s)//2)

        res = float('inf')
        flips1, flips2 = 0, 0
        l,r = 0,0

        while r < len(s):
            if s[r] != target1[r]:
                flips1 += 1
            if s[r] != target2[r]:
                flips2 += 1

            if (r-l+1) > strlen:
                if s[l] != target1[l]:
                    flips1 -= 1
                if s[l] != target2[l]:
                    flips2 -= 1
                l += 1

            if (r-l+1) == strlen:
                res = min(res, flips1, flips2)
            r += 1
        
        return res

s = Solution()
s.minFlips('111000')
s.minFlips('010')
s.minFlips('1110')
s.minFlips('01001001101')   

2

### Better Implementation

In [78]:
class Solution:
    def minFlips(self, s: str) -> int:
        n = len(s)
        s = s + s
        alt1, alt2 = '', ''
        for i in range(len(s)):
            alt1 += '0' if i % 2 else '1'
            alt2 += '1' if i % 2 else '0'
        
        res = len(s)
        diff1, diff2 = 0, 0
        l=0

        for r in range(len(s)):
            if s[r] != alt1[r]:
                diff1 += 1
            if s[r] != alt2[r]:
                diff2 += 1
            
            if (r-l+1) > n:
                if s[l] != alt1[l]:
                    diff1 -= 1
                if s[l] != alt2[l]:
                    diff2 -= 1
                l += 1

            if (r-l+1) == n:
                res = min(res, diff1, diff2)
        
        return res

In [85]:
s = Solution()
s.minFlips('111000')
s.minFlips('010')
s.minFlips('1110')
s.minFlips('01001001101')

2

### Your shit implementation

- Gets TLE, but the idea is there I think

In [70]:
class Solution:
    def minFlips(self, s: str) -> int:
        
        ## Create targets (the alternating binary string)
        target1, target2 = '01' * (len(s)//2 + 1), '10' * (len(s)//2 + 1)
        target1 = target1[:len(s)]
        target2 = target2[:len(s)]

        ## Create array to store matches compared to target1 and target2
        flips1_arr, flips2_arr = [], []
        
        ## Create variable to store count of flips
        flips1, flips2 = 0, 0
        res = float('inf')

        ## Looping over s and the 2 targets, create your flips array comparing to target1 and target2
        for c, t1, t2 in zip(s, target1, target2):
            if c != t1:
                flips1_arr.append(1)
                flips1 += 1
            else:
                flips1_arr.append(0)
            
            if c != t2:
                flips2_arr.append(1)
                flips2 += 1
            else:
                flips2_arr.append(0)

        ## Set result to the min of the current result, and the count of flips for this string
        res = min(res, flips1, flips2)
        if res == 0:
            return res
        
        ## Next, we want to iterate over s, moving the first character to the end N-1 times. The n-th time just gives you the original string, so we skip it
        _s = s

        for _ in range(len(s)-1):
            ## Cast string after type1 operation
            _s = _s[1:] + _s[0]
            
            flips1, flips2 = (
                flips2 - flips2_arr[0] + (_s[-1] != target1[-1]), 
                flips1 - flips1_arr[0] + (_s[-1] != target2[-1])
            )
            flips1_arr.append(1 if (_s[-1] != target1[-1]) else 0)
            flips2_arr.append(1 if (_s[-1] != target2[-1]) else 0)
            flips1_arr, flips2_arr = flips2_arr[1:], flips1_arr[1:]
            
            res = min(res, flips1, flips2)
        
        return res

In [71]:
s = Solution()
s.minFlips('111000')
s.minFlips('010')
s.minFlips('1110')

1

In [72]:
s.minFlips('01001001101')
# s.minFlips('10010011010')
# s.minFlips('00100110101')
# s.minFlips('01001101010')

2